Persona hub generates sample personas using documents from RedPajamav2 dataset: They prompt GPT4o with a document from the dataset and prompt it thus:

*Who is likely to read/like/dislike the following text? Be very detailed in your description of this persona*

They then use those personas to generate 'diverse' instructions. **But are the synthesized instructions actually that diverse?** Analyzing that in this notebook

In [1]:
import pandas as pd
import random
import numpy as np
from tqdm import tqdm
import ipdb
import re
from datasets import load_dataset
from diversity import compression_ratio, homogenization_score, ngram_diversity_score, extract_patterns, get_pos, pos_patterns, token_patterns
import json

import matplotlib.pyplot as plt
# import mplcursors
import seaborn as sns
%matplotlib inline
sns.set(style='darkgrid', context='notebook', rc={'figure.figsize':(14,10)}, font_scale=2)

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('chained_assignment',None)

# Set random seeds for reproducibility on a specific machine
random.seed(1)
np.random.seed(1)
np.random.RandomState(1)
np.set_printoptions(precision=3)

In [2]:
df = pd.read_json("hf://datasets/proj-persona/PersonaHub/knowledge.jsonl", lines=True)

In [3]:
df.shape

(10000, 3)

In [11]:
df['title'] = df['synthesized text'].apply(lambda x: x.split('\n\n')[0][7:])
df.head()

input persona  \
0                                                                                                                                                                           A high school teacher of social studies preparing lessons on the development of political parties in the United States.   
1                                                                                                                                                                                                              A geologist studying sunken volcanoes and the geological history of Pacific islands.   
2                                                                         A manuscript studies scholar or paleographer with expertise in early medieval European texts and handwriting. This expert would be intrigued by the ninth-century manuscript that contains copies of Theuthild's letters.   
3  I'm a legal scholar specializing in attorney conduct, courtroom behavior, and the First Amendment implications of disciplinary actions against lawyers. I'm interested in analyzing the Snyder case and its impact on attorneys' freedom of speech in their communications with court personnel.   
4                                                                                                                                                             A linguist specialized in the Khoekhoe language, also known as naq, with interest in endangered languages and language documentation.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

# How diverse are the titles?

In [18]:
titles = df['title'].values.tolist()
cr = compression_ratio(titles, 'gzip')
# hs = homogenization_score(titles, 'rougel')
# hs = homogenization_score(titles, 'bertscore') 
nds = ngram_diversity_score(titles, 4)

print(cr, nds)

3.238 2.204


Let's compare it to something standard as data - CNN/DailySum maybe?

The essays from Visakh/He He's paper showing that GPT-2 text is homogenous might have been a good start, but its too narrow and not big enough.

In [20]:
events = []
with open('../data/essays_solo.jsonl', 'r') as f:
    for event in f:
        events.append(json.loads(event))

In [23]:
essay_titles = [x['title'] for x in events]
cr = compression_ratio(essay_titles, 'gzip')
# hs = homogenization_score(titles, 'rougel')
# hs = homogenization_score(titles, 'bertscore') 
nds = ngram_diversity_score(essay_titles, 4)

print(cr, nds)

11.49 0.437


# How diverse are the generated knowledge base articles?

In [25]:
def clean_up(text):

    # remove title first
    text = " ".join(text.split('\n\n')[1:])

    return text

df['content'] = df['synthesized text'].apply(lambda x: clean_up(x))

In [28]:
contents = df['content'].values.tolist()
cr = compression_ratio(contents, 'gzip')
# hs = homogenization_score(data_example, 'rougel')
# hs = homogenization_score(data_example, 'bertscore') 
nds = ngram_diversity_score(contents, 4)

print(cr, nds)

3.06 1.72


In [30]:
essay_bodies = [x['essay'] for x in events]
cr = compression_ratio(essay_bodies, 'gzip')
# hs = homogenization_score(data_example, 'rougel')
# hs = homogenization_score(data_example, 'bertscore') 
nds = ngram_diversity_score(essay_bodies, 4)

print(cr, nds)

2.901 2.738


Even compared to the human written essays, the contents are more easily compresssed and have a higher n-gram diversity score. 

**What about POS CR?**

In [32]:
synt_patterns = token_patterns(contents, 5)

In [34]:
synt_patterns

[("In this article, I'll share", 3221),
 ("I've had the privilege of", 2548),
 ("this article, I'll share my", 2483),
 ("this article, I'll delve into", 1510),
 ("In this article, I'll delve", 1499),
 ('In this article, I will', 1485),
 ("article, I'll delve into the", 1404),
 ("article, I'll share my insights", 1254),
 ('I hope this article has', 1234),
 ('a deeper understanding of the', 1209)]

In [35]:
# get the POS patterns 
joined_pos, tuples = get_pos(contents)
ngrams_pos = token_patterns(joined_pos, 5, 10)

In [38]:
ngrams_pos

[('. LS LS NFP NFP', 38990),
 ('NN . LS LS NFP', 18479),
 ('LS LS NFP NFP NNP', 15855),
 ('NNS . LS LS NFP', 15820),
 ('NN , NN , CC', 15106),
 ('NNS , NNS , CC', 13133),
 ('NN , PRP MD VB', 12115),
 ('DT JJ NN IN DT', 12035),
 ('LS LS NFP NFP NN', 11149),
 ('IN DT NN , PRP', 10743)]

In [39]:
cr = compression_ratio(joined_pos, 'gzip')
# hs = homogenization_score(data_example, 'rougel')
# hs = homogenization_score(data_example, 'bertscore') 
nds = ngram_diversity_score(joined_pos, 4)

print(cr, nds)

6.089 0.02


In [41]:
# Now for essays

# get the POS patterns 
joined_pos, tuples = get_pos(essay_bodies)
ngrams_pos = token_patterns(joined_pos, 5, 10)

In [42]:
cr = compression_ratio(joined_pos, 'gzip')
# hs = homogenization_score(data_example, 'rougel')
# hs = homogenization_score(data_example, 'bertscore') 
nds = ngram_diversity_score(joined_pos, 4)

print(cr, nds)

5.072 0.518


| Source | NDS (Lexical) | CR-Lexical | CR-POS |
| --- | --- | --- | --- |
| Human solo essays | 2.74 | 3.06 | 5.07 | 
| Personahub-essays | 1.72 | 2.90 | 6.09 | 

## Persona hub synthesized instructions versus Dolly

In [2]:
from datasets import load_dataset
import numpy as np

In [3]:
personahub = load_dataset("proj-persona/PersonaHub", "instruction")['train'].to_pandas()

In [12]:
from diversity import compression_ratio, homogenization_score, ngram_diversity_score, extract_patterns, get_pos, pos_patterns, token_patterns, self_repetition_score

def diversity_metrics(sents):
    cr = compression_ratio(sents, 'gzip')
    print(f"CR: {np.round(cr,2)}")

    joined_pos, _ = get_pos(sents)
    cr_pos = compression_ratio(joined_pos, 'gzip')
    print(f"CR-POS: {np.round(cr_pos,2)}")
    
    nds = ngram_diversity_score(sents, 4)
    print(f"NDS: {np.round(nds,2)}")
    
    srep = self_repetition_score(sents, verbose=True)
    print(f"Self-rep:{np.round(srep,2)}")

In [5]:
personahub

,input persona,synthesized text,description
0,"A theater manager or events coordinator interested in understanding the operational aspects, facilities, and programming of performing arts centers, such as the Broward Center.",What are the key considerations for scheduling and logistics when hosting a multi-show festival at a performing arts center like the Broward Center?,2-shot instruction data synthesis
1,An urban planner looking to understand the distribution and organization of public services in the Halifax Regional Municipality.,"Compare and contrast the distribution of public services such as libraries, community centers, and public transportation in different neighborhoods of Halifax, and discuss how the municipal government's urban planning strategies impact access to these services for residents of varying socioeconomic backgrounds.",2-shot instruction data synthesis
2,A high school literature teacher looking for supplementary materials to enrich their curriculum and provide students with a deeper understanding of their state's cultural heritage.,"Can you provide me with a list of 5-7 educational resources, including videos, articles, and interactive websites, that explore the literary works and cultural significance of Native American authors from my state, such as Louise Erdrich or Sherman Alexie, to incorporate into my 11th-grade American Literature curriculum and help my students better understand the cultural heritage of our region?",0-shot instruction data synthesis
3,"A science fiction writer, exploring the use of secret trials as a theme in stories that examine surveillance, privacy, and power dynamics in society.","In a dystopian future, a young journalist named Maya discovers a hidden underground network of secret trials, where the government uses advanced surveillance technology to manipulate and control citizens. As she delves deeper into the truth, she realizes that her own sister was a victim of these trials and begins to question the true extent of the government's power. How can Maya use her skills as a journalist to expose the secret trials and bring justice to those affected, while navigating the treacherous landscape of government surveillance and propaganda?",2-shot instruction data synthesis
4,"A financial analyst specializing in Asian markets and wealthy individuals, interested in tracking the investments and philanthropic activities of billionaires like Gerald Chan.","Gerald Chan's investment firm has recently acquired a significant stake in a Chinese renewable energy company. How might this investment impact the firm's overall portfolio, and what potential philanthropic opportunities might arise from this acquisition, particularly in the context of China's growing focus on sustainable development?",2-shot instruction data synthesis
...,...,...,...
49995,An events planner seeking unique and engaging entertainment options for corporate gatherings and private parties.,"Can you suggest 5 unconventional entertainment ideas that can be tailored to a tech company's annual gala, with a focus on interactive experiences that encourage team building and networking among 200 guests?",0-shot instruction data synthesis
49996,"An music historian specializing in the analysis of librettos and their cultural significance, with a focus on 20th century dramatic oratorios.","Can you provide a comparative analysis of the librettos of William Walton's ""Belshazzar's Feast"" (1931) and Michael Tippett's ""A Child of Our Time"" (1939-1941), exploring how their use of biblical narratives reflected the cultural and political anxieties of interwar Britain?",0-shot instruction data synthesis
49997,"A science communicator who enjoys sharing stories about space exploration and the people behind it, aiming to inspire interest in science and technology among general audiences.","Can you help me write a script for a 5-minute video about the life and achievements of NASA astronaut Sally Ride, highlighting her contributions to sp

In [6]:
dolly = load_dataset("databricks/databricks-dolly-15k")["train"].to_pandas()

In [7]:
dolly.shape

(15011, 4)

In [8]:
personahub.shape

(50000, 3)

In [9]:
diversity_metrics(dolly.instruction.values.tolist())

Calculating self-repetition score: 100%|██████████████████████████████████████████████████| 15011/15011 [00:00<00:00, 417702.38it/s]

CR: 2.58
CR-POS: 5.82
NDS: 2.33
Self-rep:1.95


In [13]:
diversity_metrics(personahub['synthesized text'].sample(15000).values.tolist())

CR: 2.84
CR-POS: 6.2
NDS: 2.0


Calculating self-repetition score: 100%|██████████████████████████████████████████████████| 15000/15000 [00:00<00:00, 124771.31it/s]

Self-rep:5.13


In [14]:
nr = load_dataset("HuggingFaceH4/no_robots")['train'].to_pandas()
nr.shape

(9500, 4)

In [15]:
np.median([len(x) for x in nr['prompt'].values.tolist()])

np.float64(157.0)

In [16]:
diversity_metrics(nr['prompt'].values.tolist())

CR: 2.47
CR-POS: 5.13
NDS: 2.44


Calculating self-repetition score: 100%|█████████████████████████████████████████████████████| 9500/9500 [00:00<00:00, 90555.54it/s]

Self-rep:2.62


In [17]:
tulu = load_dataset("allenai/tulu-3-sft-personas-instruction-following")['train'].to_pandas()
tulu.shape

(29980, 4)

In [18]:
diversity_metrics(tulu['prompt'].values.tolist())

CR: 3.2
CR-POS: 6.3
NDS: 1.51


Calculating self-repetition score: 100%|██████████████████████████████████████████████████| 29980/29980 [00:00<00:00, 108111.12it/s]


Self-rep:6.79


## Math

In [30]:
gsm8k = load_dataset("openai/gsm8k", "main")['train'].to_pandas()
gsm8k.shape

(7473, 2)

In [31]:
diversity_metrics(gsm8k['question'].values.tolist())

CR: 2.87
CR-POS: 5.94
NDS: 2.19


Calculating self-repetition score: 100%|████████████████████████████████████████████████████| 7473/7473 [00:00<00:00, 236848.33it/s]

Self-rep:2.88


In [24]:
tulu_math = load_dataset("allenai/tulu-3-sft-personas-math-grade")['train'].to_pandas()
tulu_math.shape

README.md:   0%|          | 0.00/749 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/52.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/49980 [00:00<?, ? examples/s]

(49980, 3)

In [25]:
diversity_metrics(tulu_math['prompt'].sample(10000).values.tolist())

CR: 3.11
CR-POS: 6.26
NDS: 1.87


Calculating self-repetition score: 100%|███████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 75599.47it/s]

Self-rep:5.68


In [32]:
persona_math = load_dataset("proj-persona/PersonaHub", "math")['train'].to_pandas()
persona_math.shape

math.jsonl:   0%|          | 0.00/48.7M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

(50000, 3)

In [33]:
diversity_metrics(persona_math['synthesized text'].sample(10000).values.tolist())

CR: 3.11
CR-POS: 5.92
NDS: 1.75


Calculating self-repetition score: 100%|███████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 37335.57it/s]

Self-rep:6.38
